# Concordance Analysis: SPECT/CT + Sestamibi Negative, 4D CT Positive

### Objective

Evaluate **4D CT concordance with surgical findings** in patients with **non-localizing SPECT/CT and/or Sestamibi but positive 4D CT scans**.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging and surgical outcomes)

### Outputs

- **Total cases:** 50
- **Exact match:** 24 (**48.0%**)
- **Side-only match:** 9 (**18.0%**)
- **Combined side match:** 33 (**66.0%**)
- **Incorrect localization:** 17 (**34.0%**)
---

In [42]:
import pandas as pd 
import matplotlib.pyplot as plt
preop_imaging_df = pd.read_excel('../../Data/final_cleaned_with_hyperplasia_concordance.xlsx')


In [ ]:

# Define concordance function for 4D CT Scan with correct 'No Scan' and hyperplasia handling
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical Findings']

    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Handle 4 gland hyperplasia: ALWAYS incorrect
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    # Treat 'No Scan' and 'Non-localizing' distinctly
    if imaging == 'No Scan':
        return 'No Scan'
    if imaging == 'Non-localizing':
        return 'Non-localizing'

    # Exact gland match
    imaging_glands = [g.strip().lower() for g in imaging.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]

    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'

    # Side match (if laterality overlaps)
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'

    return 'Incorrect Localization'


In [44]:
# Apply concordance calculation
preop_imaging_df['4D CT Scan Calculated Concordance'] = preop_imaging_df.apply(
    lambda row: check_concordance(row, '4D CT Scan'),
    axis=1
)


# Define function to combine SPECT/CT and Sestamibi results
def check_spect_sestamibi(row):
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']

    spect_localizing = pd.notnull(spect) and spect not in ['No Scan', 'Non-localizing']
    sestamibi_localizing = pd.notnull(sestamibi) and sestamibi not in ['No Scan', 'Non-localizing']

    if not pd.notnull(spect) and not pd.notnull(sestamibi):
        return 'No SPECT/CT or Sestamibi Recorded'
    elif spect_localizing and sestamibi_localizing:
        return f"{spect}, {sestamibi}"
    elif spect_localizing:
        return spect
    elif sestamibi_localizing:
        return sestamibi
    else:
        return 'Non-localizing'

In [45]:

# Apply combined imaging results column
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(check_spect_sestamibi, axis=1)


# Filter for SPECT/CT + Sestamibi negative but 4D CT positive cases
negative_spectmibi = preop_imaging_df[preop_imaging_df['SPECT/CT-Sestamibi'] == 'Non-localizing']
sestamibi_neg_4d_pos = negative_spectmibi[
    negative_spectmibi['4D CT Scan'].notnull() & 
    (negative_spectmibi['4D CT Scan'] != 'Non-localizing')
]

# Calculate counts for each concordance category
total_cases = sestamibi_neg_4d_pos.shape[0]
exact_count = (sestamibi_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Exact)').sum()
side_count = (sestamibi_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Side)').sum()
incorrect_count = (sestamibi_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Incorrect Localization').sum()
no_scan_count = (sestamibi_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'No Scan').sum()
non_localizing_count = (sestamibi_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Non-localizing').sum()

In [ ]:
# Calculate percentages
def calc_pct(count): return f"{(count/total_cases)*100:.2f}%" if total_cases > 0 else 'N/A'

# Create final clean summary dataframe
summary_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Side Match',
        'Incorrect Localization',
        
    ],
    'Count': [
        exact_count,
        side_count,
        incorrect_count,
    ],
    'Percentage': [
        calc_pct(exact_count),
        calc_pct(side_count),
        calc_pct(incorrect_count),
    ]
})

# Display results
print(f"Total SPECT/CT + Sestamibi negative / 4D CT positive cases: {total_cases}")
display(summary_df)


Total SPECT/CT + Sestamibi negative / 4D CT positive cases: 51


,Match Type,Count,Percentage
0,Exact Match,24,47.06%
1,Side Match,9,17.65%
2,Incorrect Localization,18,35.29%


In [37]:
# import matplotlib.pyplot as plt 
# labels = ['Exact Match', 'Side Match', 'Incorrect Localization']
# counts = [exact_count, side_count, incorrect_count]
# colors = ['green', 'orange', 'red']

# plt.figure(figsize=(6,4))
# bars = plt.bar(labels, counts, color=colors)

# for bar in bars: 
#     height = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f'{int(height)}',
#              ha='center', va='bottom', fontsize=10)
    
# plt.title('4D CT Concordance in Patients with Non-localizing SPECT/CT + Sestamibi)', fontsize=12)
# plt.ylabel('Number of Patients')
# plt.ylim(0, max(counts)*1.2)  # Adds headroom for labels

# plt.tight_layout()
# plt.show()


---

### Interpretation

4D CT provided **localization (exact or side)** in **two-thirds of patients** with non-localizing SPECT/CT + Sestamibi scans, supporting its role as a **salvage imaging modality** in challenging cases.



### Next Steps

- Compare to overall **4D CT sensitivity and PPV** to guide imaging algorithm recommendations for the manuscript.